In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2021-10-23 11:33:25--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.119.128, 108.177.111.128, 142.250.1.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   190MB/s    in 0.3s    

2021-10-23 11:33:26 (190 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import zipfile

In [3]:
unzip = zipfile.ZipFile("cats_and_dogs_filtered.zip", "r")

unzip.extractall()
unzip.close()

In [4]:
train_dir = "cats_and_dogs_filtered/train/"
test_dir = "cats_and_dogs_filtered/validation/"

##**Processing the Data**

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen = ImageDataGenerator(rescale= 1/255.)
test_gen = ImageDataGenerator(rescale= 1/255.)



In [7]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (128,128),
                                           class_mode = "binary",
                                           batch_size = 32
                                           )
test_data = test_gen.flow_from_directory(test_dir,
                                         target_size = (128,128),
                                         class_mode = "binary",
                                         batch_size = 32
                                         )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
train_data,test_data

(<keras.preprocessing.image.DirectoryIterator at 0x7f3939b97150>,
 <keras.preprocessing.image.DirectoryIterator at 0x7f3939b97a50>)

In [9]:
train_data.num_classes


2

## **Creating the Model using transfer learning**

In [10]:
# creating the Model with   Mobile Net 
import tensorflow as tf
from tensorflow.keras import layers 

base_model = tf.keras.applications.MobileNetV2(include_top= False, weights="imagenet")

base_model.trainable = False

inputs = layers.Input(shape=(128,128,3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation= "sigmoid")(x)
model = tf.keras.Model(inputs, outputs)

# compiling the Model
model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

# fitting the model

model.fit(train_data,
          epochs = 5,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
        
          )

9420800/9406464 [==============================] - 0s 0us/step
Epoch 1/5
63/63 [==============================] - 43s 180ms/step - loss: 0.3208 - accuracy: 0.8535 - val_loss: 0.1976 - val_accuracy: 0.9210
Epoch 2/5
63/63 [==============================] - 10s 153ms/step - loss: 0.1567 - accuracy: 0.9360 - val_loss: 0.1968 - val_accuracy: 0.9140
Epoch 3/5
63/63 [==============================] - 10s 153ms/step - loss: 0.1293 - accuracy: 0.9460 - val_loss: 0.1834 - val_accuracy: 0.9250
Epoch 4/5
63/63 [==============================] - 10s 153ms/step - loss: 0.1154 - accuracy: 0.9580 - val_loss: 0.1845 - val_accuracy: 0.9240
Epoch 5/5
63/63 [==============================] - 10s 157ms/step - loss: 0.1056 - accuracy: 0.9590 - val_loss: 0.1917 - val_accuracy: 0.9230


In [11]:
model_evl = model.evaluate(test_data)

print(f"Model Accuracy: {model_evl[1] * 100: .2f} %")

32/32 [==============================] - 3s 105ms/step - loss: 0.1917 - accuracy: 0.9230
Model Accuracy:  92.30 %


##**Fine Tuning the model**

In [12]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, None, 3 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [13]:
len(base_model.layers)

154

In [16]:
# tuning the first 100 layers of the model

base_model.trainable = True

for layer in base_model.layers[:-5]:
  layer.trainable = False

# compiling the model

model.compile(
    loss= tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
    metrics = ["accuracy"]
)


# fitting the model

model.fit(train_data,
          epochs = 5,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
          )

Epoch 1/5
63/63 [==============================] - 14s 175ms/step - loss: 0.0437 - accuracy: 0.9840 - val_loss: 0.2023 - val_accuracy: 0.9310
Epoch 2/5
63/63 [==============================] - 10s 157ms/step - loss: 0.0303 - accuracy: 0.9915 - val_loss: 0.2213 - val_accuracy: 0.9330
Epoch 3/5
63/63 [==============================] - 10s 158ms/step - loss: 0.0192 - accuracy: 0.9960 - val_loss: 0.1884 - val_accuracy: 0.9350
Epoch 4/5
63/63 [==============================] - 10s 157ms/step - loss: 0.0097 - accuracy: 0.9985 - val_loss: 0.1825 - val_accuracy: 0.9390
Epoch 5/5
63/63 [==============================] - 10s 158ms/step - loss: 0.0058 - accuracy: 0.9995 - val_loss: 0.1884 - val_accuracy: 0.9380


In [17]:
tuned_model_evl = model.evaluate(test_data)

print(f"Tuned Model Accuracy: {tuned_model_evl[1]*100 : 0.2f}%")

32/32 [==============================] - 3s 106ms/step - loss: 0.1884 - accuracy: 0.9380
Tuned Model Accuracy:  93.80%
